Requirement: run `frequency_labels_binning.ipynb` first to generate the labels this experiment works with.

### Objectives
- Estimate word level RL using frequency data only by cumulative binning

In [31]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import pandas as pd

with open('../data/freq/all_levels_freq_binning_cumulative.pkl', 'rb') as f:
  words_and_levels_freq_binning_cum = pickle.load(f)

def get_rl(token, model, oov_level = 3):
  try:
      return model[token]
  except:
      return oov_level

## testing data in fragments
frag_train = pd.read_csv('../data/all_train_aligned.csv')
frag_dev = pd.read_csv('../data/all_dev_aligned.csv')
frag_test = pd.read_csv('../data/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]

def levels_pipeline(fragment, model):
  tokens = [t.split('#')[0] for t in fragment.split(' ')]
  gt = [t.split('#')[1] for t in fragment.split(' ')]

  # decision round 1:
  levels = [get_rl(token, model) for token in tokens]

  return {
      'levels': levels,
      'gts': gt,
  }

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
frag_dev

,Unnamed: 0,0
0,0,إبراهيم#3 الكاتب#3
1,1,إبراهيم#3 عبد#3 القادر#3 المازني#3
2,2,الفصل#3 الأول#3
3,3,وكان#3 مساء#3
4,4,شوشو#3 فتاة#3 يقول#3 لك#3 جسمها#3 إنها#3 ناهزت...
...,...,...
2962,2983,فإلى#3 الغد#3 في#3 مثل#3 هذا#3 الوقت#3
2963,2984,فوقفوا#3 جميعا#3
2964,2985,وقالوا#3 بصوت#3 واحد#3 حفظ#3 الله#3 الملك#3 ال...
2965,2986,ثم#3 انصرفوا#3


In [32]:
xx = list(words_and_levels_freq_binning_cum.keys())

In [33]:
results = [
    [levels_pipeline(f, words_and_levels_freq_binning_cum[w][0]) for f in frag_dev['0']]
    for w in xx
]



In [34]:
import numpy as np
from sklearn.metrics import classification_report

In [35]:
res = [
    np.concatenate([e['levels'] for e in r])
    for r in results
]

final_gt = np.concatenate([e['gts'] for e in results[0]])

final_gt = final_gt.astype(int)

def results_to_csv(result_arr):
  all_rows = []
  for resu in result_arr:
    inv_report = classification_report(final_gt, resu, output_dict = True)

    arr_inv = np.concatenate([[inv_report[x]['f1-score'],
            inv_report[x]['precision'],
            inv_report[x]['recall'],] for x in ['3', '4', '5']])
    arr_inv = np.append(arr_inv, inv_report['accuracy'])
    arr_inv = np.append(arr_inv, inv_report['macro avg']['f1-score'])

    all_rows.append(arr_inv)

  return all_rows

In [36]:
pd.DataFrame(results_to_csv(res), columns= ['f1_3','3_prec','3_recall','f1_4','4_prec','4_recall','f1_5','5_prec','5_recall','accuracy','f1_macro'])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

,f1_3,3_prec,3_recall,f1_4,4_prec,4_recall,f1_5,5_prec,5_recall,accuracy,f1_macro
0,0.930297,0.883305,0.982570,0.214610,0.563991,0.132518,0.147445,0.279006,0.100198,0.866727,0.430784
1,0.930464,0.877680,0.990003,0.147178,0.644269,0.083078,0.129688,0.305147,0.082341,0.867950,0.402443
2,0.928979,0.870273,0.996179,0.060934,0.849315,0.031600,0.071867,0.308271,0.040675,0.866818,0.353927
3,0.927796,0.867246,0.997435,0.000000,0.000000,0.000000,0.043282,0.237624,0.023810,0.864326,0.323693
4,0.927878,0.865459,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.865459,0.309293
5,0.927878,0.865459,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.865459,0.309293
6,0.927878,0.865459,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.865459,0.309293
